<a href="https://colab.research.google.com/github/hso0321/PythonExam/blob/master/Cifar10WithCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

In [0]:
# cifar10 데이터 셋을 다운로드 받기 위해, keras load_data 함수를 import
from tensorflow.keras.datasets.cifar10 import load_data

# Cifar10 데이터 셋을 다운로드하고 불러온다
(x_train, y_train), (x_test, y_test) = load_data()

# train 데이터의 이미지 배열과 레이블의 shape를 확인
print(x_train.shape)
print(y_train.shape)

# test 데이터의 이미지 배열과 레이블의 shape를 확인
print(x_test.shape)
print(y_test.shape)

In [0]:
plt.figure(figsize=(3, 3))
plt.imshow(x_train[17])
plt.show()

In [0]:
# 다음 배치를 읽어오기 위한 next_batch 함수를 정의한다.
def next_batch(num, data, labels):
  idx = np.arange(0,len(data))
  np.random.shuffle(idx)
  idx = idx[:num]
  data_suffle = [data[i] for i in idx]
  labels_suffle = [labels[i] for i in idx]

  return np.asarray(data_suffle), np.asarray(labels_suffle)

In [0]:
# CNN 모델을 정의한다.
def build_CNN_classifier(x):
  x_image = x

  # 1st conv2d
  # 5x5 64 filter(channel)
  # 32*32*3 -> 32*32*64
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,3,64],stddev=5e-2))      # stddev = 분산율, truncated_nomal은 random 노말 대신 들어감
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image,W_conv1, strides=[1,1,1,1], padding='SAME')+b_conv1)
  # 1st max pooling Layer
  # 입력 이미지를 1/2로 다운샘플링
  # 32*32*64 -> 16*16*64
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

  # 2st conv2d
  # 5x5 64 filter(channel)
  # 16*16*64 -> 16*16*64
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,64,64],stddev=5e-2))      # stddev = 분산율, truncated_nomal은 random 노말 대신 들어감
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1,W_conv2, strides=[1,1,1,1], padding='SAME')+b_conv2)
  # 2st max pooling
  # 입력 이미지를 1/2로 다운샘플링
  # 16x16x64 -> 8*8*64
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

  # 3st conv2d
  # 3x3 128 filter(channel)
  # 8*8*64 -> 8*8*128
  W_conv3 = tf.Variable(tf.truncated_normal(shape=[3,3,64,128],stddev=5e-2))      # stddev = 분산율, truncated_nomal은 random 노말 대신 들어감
  b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2,W_conv3, strides=[1,1,1,1], padding='SAME')+b_conv3)

  # 4st conv2d
  # 3x3 128 filter(channel)
  # 8*8*128 -> 8*8*128
  W_conv4 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128],stddev=5e-2))      # stddev = 분산율, truncated_nomal은 random 노말 대신 들어감
  b_conv4 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3,W_conv4, strides=[1,1,1,1], padding='SAME')+b_conv4)

  # 5st conv2d
  # 3x3 128 filter(channel)
  # 8*8*128 -> 8*8*128
  W_conv5 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128],stddev=5e-2))      # stddev = 분산율, truncated_nomal은 random 노말 대신 들어감
  b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4,W_conv5, strides=[1,1,1,1], padding='SAME')+b_conv5)

  # Fully connected Layer
  # 8*8*128(8196) -> 384 (Feature)
  # 8*8 크기를 가진 128개의 activation map을 1024개의 feature로 변환
  W_fc1 = tf.Variable(tf.truncated_normal(shape=[8*8*128,1024],stddev=5e-2))      # stddev = 분산율, truncated_nomal은 random 노말 대신 들어감
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
  h_pool_flat = tf.reshape(h_conv5,[-1,8*8*128])
  h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)

  # dropout
  h_fc_drop = tf.nn.dropout(h_fc1, keep_prob)

  # 384 -> 10 개
  W_fc2 = tf.Variable(tf.truncated_normal(shape=[1024, 10],stddev=5e-2))
  b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
  logits = tf.matmul(h_fc_drop, W_fc2) + b_fc2
  y_pred = tf.nn.softmax(logits)

  return y_pred, logits

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

# parameter
learningRate = 1e-3
epochs = 10000
batchSize = 128
displayStep = 100
testCount = 10

# scalar -> one-hot endcoding 타입으로 변환
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10), axis =1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10), axis =1)

y_pred, logits = build_CNN_classifier(x)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=logits))
train_step = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(loss)

# 학습이 끝나면 모델의 정확도를 확인한다
correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  # epoch 만큼 최적화
  for epoch in range(epochs):
    batch = next_batch(batchSize, x_train, y_train_one_hot.eval())

  # Training이 진행되는 중 train의 상태를 확인하기 위해서 validation을 하는 코드
  # 그러므로 keep_prob:
    if epoch % displayStep == 0:
      train_accuracy = accuracy.eval(feed_dict={x:batch[0],y:batch[1], keep_prob:1.0})        # 검증단계는 드랍아웃 없이
      loss_print = loss.eval(feed_dict={x:batch[0],y:batch[1], keep_prob:1.0})
      print("Epoch: %d, accuracy: %f, loss: %f" %(epoch, train_accuracy, loss_print))

    # Training을 하는 코드
    sess.run(train_step, feed_dict={x:batch[0],y:batch[1], keep_prob:0.8})

  # Training이 완료되면 테스트 데이터 셋으로 정확도를 측정한다.
  test_accuracy = 0.0
  batchSize = len(x_test) / testCount
  for i in range(testCount):
    test_batch = next_batch(batchSize, x_test, y_test_one_hot.eval())
    test_accuracy = test_accuracy + accuracy.eval(feed_dict = {x:test_batch[0], y:test_batch[1], keep_prob:1.0})
  test_accuracy = test_accuracy / testCount
  print("test_accuracy: %f" % test_accuracy)